In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
#List of all possible characters
CHARS = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ" \
        + "!\"#$%&\'()*+,-./:;—<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c"

#Will contain raw characters from the corpus
corpus = []
with open('shakespeare.txt', 'r') as f:
    for line in f:
        for char in line.strip():
            corpus.append(char)
        corpus.append('\n')

print("Total number of characters:", len(corpus))
print("\n\n")
print("First 100 characters:\n")
print(corpus[:100])

Total number of characters: 1115390



First 100 characters:

['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p', 'r', 'o', 'c', 'e', 'e', 'd', ' ', 'a', 'n', 'y', ' ', 'f', 'u', 'r', 't', 'h', 'e', 'r', ',', ' ', 'h', 'e', 'a', 'r', ' ', 'm', 'e', ' ', 's', 'p', 'e', 'a', 'k', '.', '\n', '\n', 'A', 'l', 'l', ':', '\n', 'S', 'p', 'e', 'a', 'k', ',', ' ', 's', 'p', 'e', 'a', 'k', '.', '\n', '\n', 'F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'Y', 'o', 'u']


In [3]:
#Map from character to its corresponding index
char2idx = {char : i for i, char in enumerate(CHARS)}
#Map from index to its corresponding character
idx2char = {i : char for i, char in enumerate(CHARS)}

NUM_CHARS = len(char2idx)
print("Total number of distinct chars:", NUM_CHARS)

Total number of distinct chars: 101


In [4]:
#Corpus but with indices as opposed to characters
corpus_with_indices = [char2idx[char] for char in corpus]

print("Corpus with indices:")
print(corpus_with_indices[:100])

SIZE_OF_SNIPPET = 250
#Dataset will contain 2000 random 250 character blocks from corpus
dataset = []
for _ in range(2000):
    
    snipped_start = np.random.randint(0, len(corpus_with_indices) - SIZE_OF_SNIPPET)
    snipped = corpus_with_indices[snipped_start:snipped_start + SIZE_OF_SNIPPET]
    
    dataset.append((
        torch.LongTensor(snipped[:-1]),
        torch.LongTensor(snipped[1:])
    ))

print("\nSize of dataset:", len(dataset))

X = torch.stack([xy[0] for xy in dataset])
Y = torch.stack([xy[1] for xy in dataset])

Corpus with indices:
[41, 18, 27, 28, 29, 95, 38, 18, 29, 18, 35, 14, 23, 77, 97, 37, 14, 15, 24, 27, 14, 95, 32, 14, 95, 25, 27, 24, 12, 14, 14, 13, 95, 10, 23, 34, 95, 15, 30, 27, 29, 17, 14, 27, 73, 95, 17, 14, 10, 27, 95, 22, 14, 95, 28, 25, 14, 10, 20, 75, 97, 97, 36, 21, 21, 77, 97, 54, 25, 14, 10, 20, 73, 95, 28, 25, 14, 10, 20, 75, 97, 97, 41, 18, 27, 28, 29, 95, 38, 18, 29, 18, 35, 14, 23, 77, 97, 60, 24, 30]

Size of dataset: 2000


In [5]:
#Define model
class ShakespeareGenerator(nn.Module):

    def __init__(self, embedding_size, hidden_size):

        super().__init__()

        #Size of embedding used to represent characters
        self.embedding_size = embedding_size
        
        #Size of hidden and cell state within LSTM
        self.hidden_size = hidden_size

        #Embedding module: Maps character indices to dense vector representations
        self.embedding = nn.Embedding(
            num_embeddings=NUM_CHARS,
            embedding_dim=self.embedding_size
        )
        
        #LSTM module to be used for character generation
        self.lstm = nn.LSTM(
            input_size=self.embedding_size,
            hidden_size=self.hidden_size
        )
        
        #Linear mapping to be used to go from LSTM outputs to character predictions
        self.linear = nn.Linear(
            in_features=self.hidden_size,
            out_features=NUM_CHARS
        )


    def forward(self, batched_inputs):
        #Number of character blocks to be considered simultaneously
        batch_size = batched_inputs.shape[1]
        #Hidden and Cell state initialized to all ones
        h, c = self.get_initial_hc(batch_size)
        #Character block length
        seq_len = batched_inputs.shape[0]

        #Embeddings from raw character inputs
        embeddings = self.embedding(batched_inputs)
        
        #Outputs and final state of LSTM after processing embedddings
        outputs, (h, c) = self.lstm(
                embeddings.reshape(seq_len, batch_size, self.embedding_size),
                (h, c)
        )
        
        #Use linear mapping to map LSTM outputs to character predictions
        outputs = self.linear(torch.squeeze(outputs))

        #Return outputs and final state
        return outputs, (h, c)


    def get_initial_hc(self, batch_size):

        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))


    def generate(self, initial_token=' ', num_tokens=100, temperature=1):
        
        with torch.no_grad():
            
            #Index of current character initialized to initial character
            token = torch.LongTensor([char2idx[initial_token]])
            #state of LSTM initialized to all ones
            h, c = self.get_initial_hc(1)
            #To contain predicted characters in a list
            chars = []
            
            for _ in range(num_tokens):
                
                #Add current character to list
                chars.append(idx2char[token.item()])
                
                #Use embedding of current character as input
                inp = self.embedding(token)
                
                #Pass current embedding through LSTM and get output and new state
                out, (h, c) = self.lstm(inp.reshape(1, 1, self.embedding_size), (h, c))
                
                #Distribution of possible character predictions based on output
                dist = self.linear(out.reshape(1, -1))
                
                #Temperature controls variation of distribution.  High temperature implies
                #likely characters are made more likely.  Low temperature increases chances
                #of less likely characters
                dist = dist.data.view(-1).div(temperature).exp()
                
                #Sample character from distribution
                chosen_i = torch.multinomial(dist, 1)[0]
                
                #Update current character
                token = torch.LongTensor([chosen_i])
                
            #Join elements of list into single string    
            return ''.join(chars[1:])


In [6]:
#Training of this model takes a long time
#For Demo we will used pretrained weights
EPOCHS = 500
LR = 0.1
BETA = 0.8
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 64

USE_PRETRAINED = True

net = ShakespeareGenerator(EMBEDDING_SIZE, HIDDEN_SIZE).float()

#Softmax Cross Entropy Loss used 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=BETA)

if USE_PRETRAINED:
    net.load_state_dict(torch.load('shakespeare.pt', map_location=lambda storage, loc: storage))
    
else:
    for _ in range(EPOCHS):

        output, _ = net(X.transpose(0, 1))
        output = output.transpose(0, 1)

        loss = criterion(output.reshape(-1, NUM_CHARS), Y.reshape(-1))

        print(loss.item())
        net.zero_grad()
        loss.backward()
        optimizer.step()
    

In [7]:
#Temperature=1 Probability Distribution used as predicted.
print(net.generate(temperature=1, num_tokens=1000))

HESER SIVENRY:
Asse.

QUEEN ELIA:
Who nacles, lews a hawnce rowse, I will mine news.

GLOUCESTER:
I, bid, of my your wooure still break I will
non.

AUFESAUMERMILAUEN:
The can a but rapharoun!
As tor,
When het he from repeptlemans cries doth palthose I
In his kneed never advoo.
If poss'd hantay, say, Pomar?
Now,
The so.

Peder for my chame hay the readed:
I heques which give hous! Sharance, thee he as your dut under of to dangidl! poss
I ckions, heefmed seady mine.-
Oxford my coussinion, of the sop shose itses.
The and a readed sacy, and
'The woinge the are adven of a foot thorred;
God-Harny'd will pread privioes, time say, sheep rect your did made.

DUKE
Haruntruconqupias if igeth than forberve
And be's mine osablandect all city and as unne
Bear beent.

Third, when to God,
And we now, stazed.

SICINIUS:
Poaten just and now.

KING HENRY VI:
Romeongs,
The lord, that not thou
ecous one.

Sray to
alied,
The let with and that for by forrHarciince unteen. Ford, thee hithall I parttherse, p


In [8]:
#Temperature=1.5 more likely characters are used more often
print(net.generate(temperature=1.5, num_tokens=1000))

tqual;
Wheirs!
Our let loro! Is; thee heavequil! evenme. Six jut: Geraick, Musimits
aticiol.
But!'; I natted':
UNIO:
So of stilling agess Abs, my dades worn,
I' am quald;
nin; thy horss
On makat--
Wirmides,
Nod, sel esuor
s becork!
Diss
Yarely hearffeth untians,, but penot:
Woum gesep? than's quickrent, athas hirst,
Nou is osting.
Why, ligh hor,
Dnors
Mell that noth he elful, frurn.
Now ish'd tebud, cut yoleest, him,
Felpex will, uld's, ose 't I it, thy fion terxmisder,
Clounmatiou, 'tioumins bah!

GLOUCIO:
Alan's: Buere's favy.

NUUHELIZAUMONEISORWBINSAUNE:
Hayour say:
If wiscence, wa, Jo!
Their ary!
wrock other! Towe. Mel gitter we liays;
Dine
That whern no
pll,
Not, past of raice
Ally sistety, both:
Apt I evesttacollewaiim;
For hikers, Has be you dat; gear meefat with buiding is main hermioublootuwn, her? I me; Your-Wa litlanged
His or my parm beef?
Gabity,-nike
I', yiliomes
Upon affair-luck, nevish prick'! KAMINA:
3 griecure your hals I vive no gepmending. Yet sink'd
Get not muter


In [9]:
#Temperature=0.25 Less likely characters are used more often.
print(net.generate(temperature=0.25, num_tokens=1000))

my shall the singer a shall the so man the seemence the man the grace the shall to the lords, be the countress and the son the so pardess the the say, the shall the words the so see the the shall be me the so present the sea the distinger death and the sea the prise and the with the shall the so be the soldies the see the his we will me the say the sea the seemence the father, the lords and the stream the see with the so man the seen the sea the be with the shall shall the son the sea the and the promised the father in the so man the shall the see the so strease he will the singer to the present the seemen of the will in the so strength the see a do the heart a so to the see the death the sea the man the live the heart the see the sea the heart you the sea the do the heart and the heart you the live my child the sea the sea the so the lords and a sea the shall the death the sea the wars the live he hath the look the see the come a pray and the son, the grace the sea the souly the so m
